In [22]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# PCA
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


In [49]:
# tabs_names = ['Ignition', 'TVB1', 'TVB2', 'TVB3', 'TVB4', 'SC', 'WDC']
tabs_names = ['mean_Ignition', 'TVB1', 'TVB2', 'TVB3', 'TVB4', 'SC', 'WDC']

### Normalization method 1: PCA

In [51]:
output_file = '/Users/yilewang/workspaces/data4project/prediction_project/block_normalized_pca.xlsx'
# with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
#     for tab_name in tabs_names:
#         tab = pd.read_excel('/Users/yilewang/workspaces/data4project/prediction_project/block_normalization.xlsx', sheet_name=tab_name, skiprows=1)
#         # do PCA
#         scaler = StandardScaler()
#         scaled_data = scaler.fit_transform(tab.iloc[:, 3:])
#         pca = PCA(n_components=1)
#         pca_result = pca.fit_transform(scaled_data)
#         # square root of the first principal component

#         # Square root of the first principal component scores
#         pca_first_component_scores = pca_result[:, 0]
#         pca_sqrt = np.sqrt(np.abs(pca_first_component_scores)).reshape(-1, 1)
        
#         # Normalize the tab using the square root of the first principal component
#         tab_sqrt = tab.iloc[:, 3:].div(pca_sqrt, axis=0)
#         # add the first three columns to the new tab
#         tab_sqrt.insert(0, 'group', tab['group'])
#         tab_sqrt.insert(1, 'caseid', tab['caseid'])
#         # write the result to each tab in a new file
#         tab_sqrt.to_excel(writer, sheet_name=tab_name, index=False)
# print(f"Data written to {output_file}")

# Open an Excel writer
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    for tab_name in tabs_names:
        # Read the data from each sheet, skipping the first row
        tab = pd.read_excel('/Users/yilewang/workspaces/data4project/prediction_project/block_normalization.xlsx', sheet_name=tab_name, skiprows=1)
        
        # Select the data columns (assuming the first three columns are not data)
        data_columns = tab.columns[3:]
        
        # Standardize the data
        scaler = StandardScaler()
        scaled_data = scaler.fit_transform(tab[data_columns])
        
        # Perform PCA to get the first principal component
        pca = PCA(n_components=1)
        pca.fit(scaled_data)
        
        # Get the first eigenvalue
        eigenvalue = pca.explained_variance_[0]
        
        # Take the square root of the first eigenvalue
        sqrt_eigenvalue = np.sqrt(eigenvalue)
        
        # Normalize the original data by dividing by the square root of the first eigenvalue
        normalized_data = tab[data_columns].div(sqrt_eigenvalue)
        # add the first three columns to the new tab
        normalized_data.insert(0, 'group', tab['group'])
        normalized_data.insert(1, 'caseid', tab['caseid'])
        # Write the result to each sheet in a new file
        normalized_data.to_excel(writer, sheet_name=tab_name, index=False)

print(f"Data written to {output_file}")


Data written to /Users/yilewang/workspaces/data4project/prediction_project/block_normalized_pca.xlsx


### Using Normalizartion method 1 to wave2 WMH data

In [65]:
def normalize_PCA_first_eigenvalues(tab):
    # Select the data columns (assuming the first three columns are not data)
    data_columns = tab.columns[5:23]
    
    # Standardize the data
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(tab[data_columns])
    
    # Perform PCA to get the first principal component
    pca = PCA(n_components=1)
    pca.fit(scaled_data)
    
    # Get the first eigenvalue
    eigenvalue = pca.explained_variance_[0]
    
    # Take the square root of the first eigenvalue
    sqrt_eigenvalue = np.sqrt(eigenvalue)
    
    # Normalize the original data by dividing by the square root of the first eigenvalue
    normalized_data = tab[data_columns].div(sqrt_eigenvalue)
    # add the first three columns to the new tab
    normalized_data.insert(0, 'group', tab['M_w2_group'])
    normalized_data.insert(1, 'caseid', tab['M_w2_ID'])
    return normalized_data

wave2_wmh = pd.read_excel('/Users/yilewang/workspaces/data4project/MAS_T2w/waves_wmh.xlsx', sheet_name='wave2')
wave2_wmh = wave2_wmh.dropna()
wave2_wmh_normalized = normalize_PCA_first_eigenvalues(wave2_wmh)
wave2_wmh_normalized.to_excel('/Users/yilewang/workspaces/data4project/prediction_project/waves2_wmh_normalized.xlsx', index=False)

### Normalization method 2: Sum of Squares

In [36]:
def normalize_sum_squares(tab):
    """
    Normalize each column in the DataFrame by calculating the sum of squares for each column,
    then dividing each value in the column by the square root of this sum.

    Args:
    tab (pd.DataFrame): The input data table with variables to be normalized.

    Returns:
    pd.DataFrame: The normalized data table.
    """
    # Initialize an empty DataFrame to store normalized data
    normalized_tab = pd.DataFrame()

    for column in tab.columns[3:]:
        # Calculate the sum of squares for the current column
        sum_of_squares = np.sum(np.square(tab[column]))
        
        # Calculate the normalization factor (square root of the sum of squares)
        normalization_factor = np.sqrt(sum_of_squares)
        
        # Normalize the current column
        normalized_tab[column] = tab[column] / normalization_factor

    return normalized_tab

tab = pd.read_excel('/Users/yilewang/workspaces/data4project/prediction_project/block_normalization.xlsx', sheet_name='All', skiprows=1)

# Normalize the data using the sum of squares method
normalized_tab = normalize_sum_squares(tab)

# Add the first three columns to the normalized data table
normalized_tab.insert(0, 'group', tab['group'])
normalized_tab.insert(1, 'caseid', tab['caseid'])

# Write the normalized data table to a new Excel file
output_file = '/Users/yilewang/workspaces/data4project/prediction_project/block_normalized_sumsquares.xlsx'
normalized_tab.to_excel(output_file, index=False)

### Normalization method 3: sqrt of N

In [44]:
def normalize_N(tab):
    # get the number of samples
    N = sum(tab.groupby('group').size())
    
    # Create a copy of the dataframe to avoid modifying the original one
    normalized_tab = tab.copy()
    
    # Iterate over the columns to normalize (assuming columns from index 3 onwards need normalization)
    # for col in tab.columns[3:]:
    #     for group in N.index:
    #         mask = tab['group'] == group
    #         sqrt_N = np.sqrt(N[group])
    #         normalized_tab.loc[mask, col] = tab.loc[mask, col] / sqrt_N
    normalized_tab.iloc[:, 3:] = tab.iloc[:, 3:].div(np.sqrt(N), axis=0)
    
    return normalized_tab

# Load the data
tab = pd.read_excel('/Users/yilewang/workspaces/data4project/prediction_project/block_normalization.xlsx', sheet_name='All', skiprows=1)

# Normalize the data
normalized_tab = normalize_N(tab)

# Save the normalized data to a new Excel file if needed
normalized_tab.to_excel('/Users/yilewang/workspaces/data4project/prediction_project/normalized_N.xlsx', index=False)